# <center>Assignment 3</center>

This assignment is based on embeddings and CNNs. You can choose to code in Python2 or Python3. All the imports made in this notebook are as below; if these imports work, you are (mostly) set to complete the assignment. You will learn the following:
* Making use of embeddings in Tensorflow
* Coding CNNs in TF
* Intuitions behind working of CNN
* Intuitions behind embeddings

In [ ]:
from __future__ import print_function,division
import random 
import tensorflow as tf
import numpy as np
import os

In [ ]:
import sys
if sys.version_info >= (3, 0):
  from builtins import map as m
  def map(f,l):
    return list(m(f,l))

## Quick Review questions

* If the input volume has dimensions 10 x 10 x 32 (Height x Width x Channels), how many weights will be there in a filter that considers an area of 5 x 5?

<b>Answer: 5x5x32.</b>
* If input volume has dimensions 10 x 10 x 32 and after convolution we get an output volume of 8 x 8 x 64, how many filters were used? 

<b>Answer: 64.</b>
* What is inverted-dropout? Why is it done? 

<b>Answer: Inverted-dropout is a way of applying dropout to neural networks by performing dropout and scaling both at training time. In contrast, normal dropout performs dropout at training time, and performs scaling at testing time. Inverted-dropout lets us to do scaling at training time, so we could leave the testing process untouched.</b>

## Sentiment Classification - dataset analysis

We will use movie review dataset taken from http://www.cs.cornell.edu/people/pabo/movie-review-data/. The exact dataset we will use is the Sentence-polarity dataset.

In [ ]:
data = []
for file_,label in zip(["class_neg.txt","class_pos.txt"],[0,1]):
    lines = open(file_).readlines()
    lines = list(map(lambda x:x.strip().replace("-"," ").split(),lines))
    for line in lines:
        data.append([line,label])
    print("Number of reviews of {} = {}".format(file_[:-4],len(lines)))
    print("\tMax number of tokens in a sentence = {}".format(max(map(lambda x:len(x),lines))))
    print("\tMin number of tokens in a sentence = {}".format(min(map(lambda x:len(x),lines))))
random.Random(5).shuffle(data)

Observe that the lengths of sentences are different. In case, we need to vectorize the operations, we need all sentences to be of equal length. Therefore, we will pad all sentences to be of equal length and substitute the padded parts of sentence with zeros. 

In [ ]:
# See some randomly sampled sentences
print(" ".join(data[random.randint(0,len(data))][0]))

We will work with the sentence as given and not remove any stop-words or punctuation marks. 

In [ ]:
sents = map(lambda x:x[0],data) # all sentences
all_words = set()
for sent in sents:
    all_words |= set(sent)
all_words = sorted(list(all_words))
vocab = {all_words[i]:i for i in range(len(all_words))}
print("Number of words : ",len(vocab))
train = data[:int(0.8*len(data))]
test = data[int(0.8*len(data)):]
train_data = []
train_targets = []
test_data = []
test_targets = []
for list_all,list_data,list_target,label_list in zip([train,test],[train_data,test_data],[train_targets,test_targets],["train","test"]):
    for datum,label in list_all:
        list_data.append([vocab[w] for w in datum])
        list_target.append([label])
    print(label_list)
    print("\tNumber of positive examples : ",list_target.count([1]))
    print("\tNumber of negative examples : ",list_target.count([0]))

For implementation purposes, we will need an index for the padded word and we will use the index 19757.
Note: For a dataset of this <i>small</i> size, we will need to do K-Fold Cross-validation to evaluate the performance. However, we will work with this train-test split for the rest of this assignment. 

## Simple Classifier

<img src="https://web.cs.dal.ca/~sastry/cnn_simple.jpg"/>

The above image shows the architecture of the simple model that we will implement for text classification. We are interested in the following hyperparameters apart from the number of filters (which we will set to 1 for this problem):
* The span of the filter/the number of words considered for making the prediction.
* The size of the stride.
* The number of activations selected for feeding into softmax classifier.

Before continuing:

* Can you reason how the machine is classifying (in the above example)? The values of the activations are color-coded. Is this the only possible way the machine can work? 

    (Your answer might look like : ... filter is ... template matching ... )

<b>Answer: The input sentence has 7 words. The dimension of the training data is set to 5, so for each word, we need to embed it as a row with 5 columns. Thus, our input data is a 7x5 matrix. The filter is of size 2, so it is a 2x5 matrix. We slide the filter from top to bottom on the input data, with a stride of 2, which will result a feature map as a 4x1 matrix. Then, we can do a pooling, which will select the 2 cells with max values, and feed them into the softmax function. After the softmax function, we get two values which will be added to 1, and we select the one with higer value, comparing it with the label of the input data, get the error, and do backpropagation.

This is not the only possible way the machine can work.</b>
    
* Why might order of activations need to be retained?

<b>Answer: It won't make a difference</b>

* In the code, we will add an additional row of zeros to represent the padded words. Will the zeros of the padded words be updated during back-prop? Why or why not?

<b>Answer: No, even though the graident will come, we will use TensorFlow's mechanisms to append zero to it.</b>

First, we will write code which can select k top elements in the order they appeared. 

In [ ]:
def k_max_pool(A,k):
    """
    A = 2 dimensional array (assume that the length of last dimension of A will be always more than k)
    k = number of elements.
    Return: For every row of A, top k elements in the order they appear.
    """
    assert len(A.get_shape())==2
    def func(row):
        """
        Hint : I used top_k and reverse.
        I am not sure whether the order of the indices are retained when sorted = False in top_k. (did not find any documentation)
        Therefore, I suggest that you sort the indices before selecting the elements from the array(Trick: use top_k again!)"""
        ret_tensor = None
        ## your code here to compute ret_tensor ##
        values, indices = tf.nn.top_k(row, k=k)
        values_, indices_ = tf.nn.top_k(indices, k=k)
        indices_ = tf.reverse(indices_, [0])
        ret_list = []
        for i in range(k):
            ret_list.append(values[indices_[i]])
        ret_tensor = tf.convert_to_tensor(ret_list)
        return ret_tensor
    return tf.map_fn(func,A)

In [ ]:
A = tf.placeholder(shape=[None,None],dtype=tf.float64)
top = k_max_pool(A,5)
sess = tf.Session()
for i in range(1,6):
    np.random.seed(5)
    l = np.random.randn(i*10,i*10)
    top_elements = sess.run(top,feed_dict={A:l})
    l = l.tolist()
    top_elements2 = np.array(map(lambda x: [x[i] for i in range(len(x)) if x[i]>sorted(x,reverse=True)[5]],l))
    # Note that this test assumes that the 6th largest element and 5th largest element are different.
    print(((top_elements - top_elements2)<10**-10).all())

In [ ]:
def initializer(shape):
    xavier = tf.contrib.layers.xavier_initializer(seed=1)
    return xavier(shape)

In [ ]:
class CNN_simple:
    def __init__(self,num_words,embedding_size = 30,span=2,k=5):
        self.num_words = num_words

        # The batch of text documents. Let's assume that it is always padded to length 100. 
        # We could use [None,None], but we'll use [None,100] for simplicity. 
        self.input = tf.placeholder(shape=[None,100],dtype=tf.int32)
        self.expected_output = tf.placeholder(shape=[None,1],dtype=tf.float32)
        

        embedding_matrix = tf.Variable(initializer((num_words, embedding_size)), name="embeddings")
        # Add an additional row of zeros to denote padded words.
        self.embedding_matrix = tf.concat([embedding_matrix, tf.zeros([1, embedding_size], dtype=tf.float32)], 0)

        
        # Extract the vectors from the embedding matrix. The dimensions should be None x 100 x embedding_size. 
        # Use embedding lookup
        vectors = tf.nn.embedding_lookup(self.embedding_matrix, self.input) # None x 100 x embedding_size
        
        # In order to use conv2d, we need vectors to be 4 dimensional.
        # The convention is NHWC - None (Batch Size) x Height(Height of image) x Width(Width of image) x Channel(Depth - similar to RGB).
        # For text, let's consider Height = 1, width = number of words, channel = embedding_size.
        # Use expand-dims to modify. 
        vectors2d = tf.expand_dims(vectors, 1) # None x 1 x 100 x embedding_size
        
        # Conv2d needs a filter bank.
        # The dimensions of the filter bank = Height, Width, in-channels, out-channels(Number-of-Filters).
        # We are creating a single filter of size = span. 
        # So, height = 1, width = span, in-channels = embedding_size ,out-channels = 1. 
        single_filter = tf.Variable(initializer((1, span, embedding_size, 1)), name="filter")  
        bias = tf.Variable(0.0,name="bias") # You need a bias for each filter.
        conv_span = tf.nn.conv2d(
            input=vectors2d,
            filter=single_filter,
            # Note that the first and last elements SHOULD be 1. 
            strides=[1, 1, 1, 1], 
            # This means that we are ok with input size being reduced during the process of convolution.
            padding="VALID"
        ) # Shape = [None, 1, 99, 1]
        
        acts = tf.nn.leaky_relu(conv_span+bias)
        
        # Now, let us extract the top k activations. 
        # But, we need to first convert acts this into 2-dimensional.  
        # Use tf.squeeze. Be sure to specify the squeeze-dimensions. 
        acts_2d = tf.squeeze(acts, [1, 3])
        
        # Use k_max_pool to extract top-k activations
        input_fully_connected = k_max_pool(acts_2d,k) # None x k
        
        # Initialize the weight and bias needed for softmax classifier. 
        softmax_weight = tf.Variable(initializer((k, 2)), name="softmax_weight")
        softmax_bias = tf.Variable(np.zeros(shape=(2)), name="softmax_bias", dtype=tf.float32)
        
        # Write out the equation for computing the logits.
        self.output = tf.nn.softmax(tf.matmul(input_fully_connected, softmax_weight) + softmax_bias, axis=1) # Shape = [None, 2]
        
        # Compute the cross-entropy cost. 
        # You might either sum or take mean of all the costs across all the examples. 
        # It is your choice as the test case is on Stochastic Training.
        self.cost = -tf.reduce_sum(self.expected_output * tf.log(self.output[:, 1]) 
                                  + (1 - self.expected_output) * tf.log(self.output[:, 0]))
        
        correct_prediction = tf.equal(tf.reshape(tf.argmax(self.output, 1),[-1,1]), tf.cast(self.expected_output, dtype=tf.int64))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))        
        
        optimizer = tf.train.AdamOptimizer()
        self.train_op = optimizer.minimize(self.cost)
        self.session = tf.Session()
        self.session.run(tf.global_variables_initializer())

    def pad(self,data,pad_word,pad_length=100):
        for datum in data:
            datum.extend([pad_word]*(pad_length-len(datum)))
        return data
    
    def train(self,train_data,test_data,train_targets,test_targets,batch_size=1,epochs=1,verbose=False):
        sess = self.session
        self.pad(train_data,self.num_words)
        self.pad(test_data,self.num_words)
        print("Starting training...")
        for epoch in range(epochs):
            cost_epoch = 0
            c = 0
            for datum,target in zip([train_data[i:i+batch_size] for i in range(0,len(train_data),batch_size)],
                                   [train_targets[i:i+batch_size] for i in range(0,len(train_targets),batch_size)]):
                _,cost = sess.run([self.train_op,self.cost],feed_dict={self.input:datum,self.expected_output:target})
                cost_epoch += cost
                c += 1
                if c%100 == 0 and verbose:
                    print("\t{} batches finished. Cost : {}".format(c,cost_epoch/c))
            print("Epoch {}: {}".format(epoch,cost_epoch/len(train_data)))
            print("\tTrain accuracy: {}".format(self.compute_accuracy(train_data,train_targets)))
            print("\tTest accuracy: {}".format(self.compute_accuracy(test_data,test_targets)))
    
    def compute_accuracy(self,data,targets):
        return self.session.run(self.accuracy,feed_dict={self.input:data,self.expected_output:targets})

In [ ]:
c=CNN_simple(len(vocab))
c.train(train_data,test_data,train_targets,test_targets,epochs=1,verbose=True)

The expected output for the above snippet is
<pre>
Starting training...
	100 batches finished. Cost : 0.688363179564
	200 batches finished. Cost : 0.695461705327
	300 batches finished. Cost : 0.695902070602
	400 batches finished. Cost : 0.697339072227
	500 batches finished. Cost : 0.698220448136
    ...
Epoch 0: 0.675099702418
	Train accuracy: 0.718958854675
	Test accuracy: 0.664322555065   
</pre>
If you get any other output and you feel you are correct, you can proceed (However, I cannot think of any case where you can get a different output). 

## ConvNet 

### Architecture

<img src="https://web.cs.dal.ca/~sastry/cnn.png" style="height:40%;width:40%">

Essentially, there are 2 kind of hyper-parameters - the filter size and number of filters of each size. In the image shown, there are 3 filter-sizes - 2,3,4 and number of filters of each size is 2. Once the convolution is obtained, 1-max pooling is done - it basically involves extracting 1 activation from the list of activations which is the maximum activation. The reason we need to do this is to construct the inputs to the softmax layer which are of a fixed size.
Read more at https://arxiv.org/pdf/1510.03820.pdf. 

In [ ]:
class CNN:
    def __init__(self,num_words,embedding_size = 30):
        self.num_words = num_words

        # The batch of text documents. Let's assume that it is always padded to length 100. 
        # We could use [None,None], but we'll use [None,100] for simplicity. 
        self.input = tf.placeholder(shape=[None,100],dtype=tf.int32)
        self.expected_output = tf.placeholder(shape=[None,1],dtype=tf.float32)
        

        embedding_matrix = tf.Variable(initializer((num_words, embedding_size)), name="embeddings")
        # Add an additional row of zeros to denote padded words.
        self.embedding_matrix = tf.concat([embedding_matrix, tf.zeros([1, embedding_size], dtype=tf.float32)], axis=0)
        
        # Extract the vectors from the embedding matrix. The dimensions should be None x 100 x embedding_size. 
        # Use embedding lookup
        vectors = tf.nn.embedding_lookup(self.embedding_matrix, self.input) # None x 100 x embedding_size
        
        # In order to use conv2d, we need vectors to be 4 dimensional.
        # The convention is NHWC - None (Batch Size) x Height(Height of image) x Width(Width of image) x Channel(Depth - similar to RGB).
        # For text, let's consider Height = 1, width = number of words, channel = embedding_size.
        # Use expand-dims to modify. 
        vectors2d = tf.expand_dims(vectors, 1) # None x 1 x 100 x embedding_size
        
        # Create 50 filters with span of 3 words. You need 1 bias for each filter.
        filter_tri = tf.Variable(initializer((1, 3, embedding_size, 50)), name="weight3")  
        bias_tri = tf.Variable(tf.zeros((1,50)), name="bias3")  
        conv1 = tf.nn.conv2d(
            input=vectors2d,
            filter=filter_tri,
            strides=[1, 1, 1, 1],
            padding="VALID"
        )  # Shape = [None, 1, words-2, 50]
        A1 = tf.nn.leaky_relu(conv1+bias_tri)

        # Create 50 filters with span of 4 words. You need 1 bias for each filter.
        filter_4 = tf.Variable(initializer((1,4,embedding_size,50)), name="weight4")  
        bias_4 = tf.Variable(tf.zeros((1,50)), name="bias4")
        conv2 = tf.nn.conv2d(
            input=vectors2d,
            filter=filter_4,
            strides=[1, 1, 1, 1],
            padding="VALID"
        )  # Shape = [None, 1, words-3, 50]

        A2 = tf.nn.leaky_relu(conv2+bias_4)

        # Create 50 filters with span of 5 words. You need 1 bias for each filter.
        filter_5 = tf.Variable(initializer((1,5,embedding_size,50)), name="weight5")  
        bias_5 = tf.Variable(tf.zeros((1,50)), name="bias5")
        conv3 = tf.nn.conv2d(
            input=vectors2d,
            filter=filter_5,
            strides=[1, 1, 1, 1],
            padding="VALID"
        )  # Shape = [None, 1, words-4, 50]

        A3 = tf.nn.leaky_relu(conv3+bias_5)
        
        # Now extract the maximum activations for each of the filters. The shapes are listed alongside. 
        max_A1 = k_max_pool(tf.reshape(tf.squeeze(A1, [1]), [-1, 98 * 50]), 50)  # None x 50
        max_A2 = k_max_pool(tf.reshape(tf.squeeze(A2, [1]), [-1, 97 * 50]), 50)  # None x 50
        max_A3 = k_max_pool(tf.reshape(tf.squeeze(A3, [1]), [-1, 96 * 50]), 50)  # None x 50
        
        max_A1 = tf.reshape(tf.squeeze(tf.reduce_max(A1, 2)), [-1, 50])  # None x 50
        max_A2 = tf.reshape(tf.squeeze(tf.reduce_max(A2, 2)), [-1, 50])  # None x 50
        max_A3 = tf.reshape(tf.squeeze(tf.reduce_max(A3, 2)), [-1, 50])  # None x 50
        
        concat = tf.concat([max_A1, max_A2, max_A3], axis=1) # None x 150
        
        # Initialize the weight and bias needed for softmax classifier. 
        self.softmax_weight = tf.Variable(initializer((150, 2)), name="softmax_weight")
        self.softmax_bias = tf.Variable(0.0 * initializer((1, 2)), name="softmax_bias")
        
        # Write out the equation for computing the logits.
        self.output = tf.nn.softmax(tf.matmul(concat, self.softmax_weight) + self.softmax_bias, axis=1) # Shape = [None, 2]
        
        # Compute the cross-entropy cost. 
        # You might either sum or take mean of all the costs across all the examples. 
        # It is your choice as the test case is on Stochastic Training. 
        self.cost = tf.reduce_sum(-self.expected_output * tf.log(self.output[:, 1]) 
                                  - (1 - self.expected_output) * tf.log(self.output[:, 0]))
        
        correct_prediction = tf.equal(tf.reshape(tf.argmax(self.output, 1),[-1,1]), tf.cast(self.expected_output, dtype=tf.int64))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        optimizer = tf.train.AdamOptimizer()
        self.train_op = optimizer.minimize(self.cost)
        self.session = tf.Session()
        self.session.run(tf.global_variables_initializer())

    def get_distance(word1,word2):
        return tf.reduce_sum(np.dot(word1, word2)) / (tf.reduce_sum(tf.sqrt(tf.square(word1))) * tf.reduce_sum(tf.sqrt(tf.square(word2))))
    
    def get_most_similar(word):
        emb = c.session.run(c.embedding_matrix)
        word_id = vocab[word]
        word_sim = {w:0 for w in vocab}
        for w2 in 
        
    def pad(self,data,pad_word,pad_length=100):
        for datum in data:
            datum.extend([pad_word]*(pad_length-len(datum)))
        return data
    
    def train(self,train_data,test_data,train_targets,test_targets,batch_size=1,epochs=1,verbose=False):
        sess = self.session
        self.pad(train_data,self.num_words)
        self.pad(test_data,self.num_words)
        print("Starting training...")
        for epoch in range(epochs):
            cost_epoch = 0
            c = 0
            for datum,target in zip([train_data[i:i+batch_size] for i in range(0,len(train_data),batch_size)],
                                   [train_targets[i:i+batch_size] for i in range(0,len(train_targets),batch_size)]):
                _,cost = sess.run([self.train_op,self.cost],feed_dict={self.input:datum,self.expected_output:target})
                cost_epoch += cost
                c += 1
                if c%100 == 0 and verbose:
                    print("\t{} batches finished. Cost : {}".format(c,cost_epoch/c))
            print("Epoch {}: {}".format(epoch,cost_epoch/len(train_data)))
            print("\tTrain accuracy: {}".format(self.compute_accuracy(train_data,train_targets)))
            print("\tTest accuracy: {}".format(self.compute_accuracy(test_data,test_targets)))
    
    def compute_accuracy(self,data,targets):
        return self.session.run(self.accuracy,feed_dict={self.input:data,self.expected_output:targets})

In [ ]:
c=CNN(len(vocab))
c.train(train_data,test_data,train_targets,test_targets,epochs=1,verbose=True)

The expected output for the above snippet is
<pre>
Starting training...
	100 batches finished. Cost : 0.692921404839
	200 batches finished. Cost : 0.694593518078
	300 batches finished. Cost : 0.695016788642
	400 batches finished. Cost : 0.695038306713
	500 batches finished. Cost : 0.693231915712
    ...
Epoch 0: 0.571991487547
	Train accuracy: 0.895532906055
	Test accuracy: 0.759962499142 
</pre>
If you get any other output and you feel you are correct, you can proceed (However, I cannot think of any case where you can get a different output). 

### Effect of Batch Size on Training

Study the effects of changing batch size. Just run the various experiments and observe the results (Run it in non-verbose mode). No need to make any comments here.

In [ ]:
c.train(train_data,test_data,train_targets,test_targets,epochs=1,batch_size=10)
c.train(train_data,test_data,train_targets,test_targets,epochs=1,batch_size=20)
c.train(train_data,test_data,train_targets,test_targets,epochs=1,batch_size=30)
c.train(train_data,test_data,train_targets,test_targets,epochs=1,batch_size=40)
c.train(train_data,test_data,train_targets,test_targets,epochs=1,batch_size=50)
c.train(train_data,test_data,train_targets,test_targets,epochs=1,batch_size=100)
c.train(train_data,test_data,train_targets,test_targets,epochs=1,batch_size=200)
c.train(train_data,test_data,train_targets,test_targets,epochs=1,batch_size=300)
c.train(train_data,test_data,train_targets,test_targets,epochs=1,batch_size=400)
c.train(train_data,test_data,train_targets,test_targets,epochs=1,batch_size=500)

### Embeddings

Add 2 functions - get_distance and get_most_similar to the CNN class (the big one). 
* get_distance(word1,word2) - should return the cosine distance between the 2 words.
* get_most_similar(word) - should return top 10 most similar words to the word passed.

Now, use the 2 functions to record the distances between a list of word-pairs as the training progresses. (One easy way to go about could be to save the embedding matrix in the hard-disk for every 5 updates.):
* Study the distance between words of opposite sentiment as the training progresses. Ex: Good and Bad, Good and horrible, etc.
* Study the distance between words of same sentiment. Ex: Good and Beautiful, Bad and Terrible, etc.
* Study how the non-sentiment bearing words relate to each other. Ex: his, her, an, it, etc

### Learnings:

List out the observations and conclusions you made from the various experiments. 